<a href="https://colab.research.google.com/github/khare19yash/Automated-Detection-of-Neuropsychiatric-Disorders/blob/master/asd_diagnet_ae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls ../input/abide-cc200/

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd gdrive/My Drive/IIITH_Internship/Neuro/Final

/content/gdrive/My Drive/IIITH_Internship/Neuro/Final


In [3]:
#options: cc200, dosenbach160, aal
p_ROI = "cc200"
p_fold = 10
p_center = "Stanford"
p_mode = "whole"
p_augmentation = True
p_Method = "ASD-DiagNet"

In [4]:
parameter_list = [p_ROI,p_fold,p_center,p_mode,p_augmentation,p_Method]
print("*****List of patameters****")
print("ROI atlas: ",p_ROI)
print("per Center or whole: ",p_mode)
if p_mode == 'percenter':
    print("Center's name: ",p_center)
print("Method's name: ",p_Method)
if p_Method == "ASD-DiagNet":
    print("Augmentation: ",p_augmentation)


*****List of patameters****
ROI atlas:  cc200
per Center or whole:  whole
Method's name:  ASD-DiagNet
Augmentation:  True


In [11]:
!pip install pyprind

In [13]:
!pip install wandb

     |████████████████████████████████| 1.8MB 4.3MB/s 
     |████████████████████████████████| 133kB 27.2MB/s 
     |████████████████████████████████| 174kB 26.1MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=bccacf31052dd4320013e6474c566bc21c2d4d7beecb48f587c80de4621c95b9
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=bdb9347df13d56806d61e4e73b69c0d405a8b3e7f8b1dc68fdc38eddceb848c7
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.impute import SimpleImputer
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import sys
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import silhouette_samples, silhouette_score
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from scipy import stats
from sklearn import tree
from sklearn.cluster import MiniBatchKMeans
import functools
import numpy.ma as ma # for masked arrays
import pyprind
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tqdm.notebook import tqdm

from itertools import groupby
import pyprind
import wandb
# !wandb login d164742a4a99e4e581f543102aff0992153ad225

## Importing the data 

In [15]:
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3]) 
    else:
        key = '_'.join(f_split[0:2])
    return key

In [16]:
cc200_data_path = './data/filt_global/rois_cc200'#cc200'#path to time series data
# flist = os.listdir(cc200_data_path)

# print(flist[0])

# fid = []
# for f in flist:
#     fid.append(get_key(f))

    
data_df = pd.read_csv('./data/Phenotypic_V1_0b_preprocessed949.csv',encoding= 'unicode_escape')#path 
# data_df = data_df[data_df['FILE_ID'].isin(fid)]
data_df.DX_GROUP = data_df.DX_GROUP.map({1: 1, 2:0})
data_df['FILE_PATH'] = data_df['FILE_ID'].apply(lambda x : os.path.join(cc200_data_path,x + '_rois_cc200.1D')) 


# print(data_df.head())
print(len(data_df))
print(data_df['FILE_PATH'].values[0])

949
./data/filt_global/rois_cc200/Pitt_0050003_rois_cc200.1D


### Helper functions for computing correlations

In [7]:
def get_label(filename):
    assert (filename in labels)
    return labels[filename]


def get_corr_data(df):
              
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        m = ma.masked_where(mask == 1, mask)
        return ma.masked_where(m, corr).compressed()


def get_corr_matrix(filename,data_path):
    # returns correlation matrix
    for file in os.listdir(data_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_path, file), sep='\t')
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        return corr

def confusion(g_turth,predictions):
    tn, fp, fn, tp = confusion_matrix(g_turth,predictions).ravel()
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    sensitivity = (tp)/(tp+fn)
    specificty = (tn)/(tn+fp)
    return accuracy,sensitivity,specificty

def get_regs(samplesnames,regnum):
    # returns region index array
    datas = []
    for sn in samplesnames:
        datas.append(all_corr[sn][0])
    datas = np.array(datas)     # datas shape len(samplesnames)x19900
    avg=[]
    for ie in range(datas.shape[1]):
        avg.append(np.mean(datas[:,ie]))
    avg=np.array(avg)
    highs=avg.argsort()[-regnum:][::-1]
    lows=avg.argsort()[:regnum][::-1]
    regions=np.concatenate((highs,lows),axis=0) # shape (9950,)
    return regions

In [17]:
labels = data_df['DX_GROUP'].values
fpaths = data_df['FILE_PATH']
sfc = []
# all_corr = {}

for i,path in enumerate(fpaths):
    corr_data = get_corr_data(path)
    sfc.append(corr_data)
#     all_corr[path] = (corr_data,labels[i])

sfc = np.asarray(sfc)

print(sfc.shape)

KeyboardInterrupt: ignored

## Defining dataset class

In [ ]:
class ASDDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        pass
    def __getitem__(self,idx):
        return torch.tensor(self.x[idx],dtype=torch.float),torch.tensor(self.y[idx],dtype=torch.float)
        pass
    def __len__(self):
        return len(self.x)
        pass
        

In [ ]:
window_size = 40
stride = window_size
f1 = sfc[:]
X = [f1[i : i + window_size] for i in range(window_size, stride)]


In [ ]:
class CC200Dataset(Dataset):
    def __init__(self, pkl_filename=None, data=None, samples_list=None, 
                 augmentation=False, aug_factor=1, num_neighbs=5,
                 eig_data=None, similarity_fn=None, verbose=False,regs=None):
        
        self.regs=regs
        if pkl_filename is not None:
            if verbose:
                print ('Loading ..!', end=' ')
            self.data = pickle.load(open(pkl_filename, 'rb'))
        elif data is not None:
            self.data = data.copy()
            
        else:
            sys.stderr.write('Either PKL file or data is needed!')
            return 

        #if verbose:
        #    print ('Preprocess..!', end='  ')
        if samples_list is None:
            self.flist = [f for f in self.data]
        else:
            self.flist = [f for f in samples_list]
        self.labels = np.array([self.data[f][1] for f in self.flist])
        
        current_flist = np.array(self.flist.copy())
        current_lab0_flist = current_flist[self.labels == 0]
        current_lab1_flist = current_flist[self.labels == 1]
        #if verbose:
        #    print(' Num Positive : ', len(current_lab1_flist), end=' ')
        #    print(' Num Negative : ', len(current_lab0_flist), end=' ')
        
        
        if augmentation:
            self.num_data = aug_factor * len(self.flist)
            self.window_size = 40
            self.stride = self.window_size
            self.neighbors = {}
            pbar = pyprind.ProgBar(len(self.flist))
            weights = norm_weights(samples_list)#??
            for f in self.flist:
                label = self.data[f][1]
                candidates = (set(current_lab0_flist) if label == 0 else set(current_lab1_flist))
                candidates.remove(f)
                eig_f = eig_data[f]['eigvecs']
                sim_list = []
                for cand in candidates:
                    eig_cand = eig_data[cand]['eigvecs']
                    sim = similarity_fn(eig_f, eig_cand,weights)
                    sim_list.append((sim, cand))
                sim_list.sort(key=lambda x: x[0], reverse=True)
                self.neighbors[f] = [item[1] for item in sim_list[:num_neighbs]]#list(candidates)#[item[1] for item in sim_list[:num_neighbs]]
        
        else:
            self.num_data = len(self.flist)

        
    def __getitem__(self, index):
        if index < len(self.flist):
            fname = self.flist[index]
            data = self.data[fname][0].copy() #get_corr_data(fname, mode=cal_mode)    
            data = data[self.regs].copy()
            label = (self.labels[index],)
            return torch.FloatTensor(data), torch.FloatTensor(label)
        else:
            f1 = self.flist[index % len(self.flist)]
            X = [f1[i : i + self.window_size] for i in range(self.window_size, self.stride)]
            return torch.FloatTensor(data), torch.FloatTensor(label)

    def __len__(self):
        return self.num_data
    
    
def get_loader(pkl_filename=None, data=None, samples_list=None,
               batch_size=64, 
               num_workers=1, mode='train',
               *, augmentation=False, aug_factor=1, num_neighbs=5,
                 eig_data=None, similarity_fn=None, verbose=False,regions=None):
    """Build and return data loader."""
    if mode == 'train':
        shuffle = True
    else:
        shuffle = False
        augmentation=False

    dataset = CC200Dataset(pkl_filename=pkl_filename, data=data, samples_list=samples_list,
                           augmentation=augmentation, aug_factor=aug_factor, 
                           eig_data=eig_data, similarity_fn=similarity_fn, verbose=verbose,regs=regions)

    data_loader = DataLoader(dataset,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             num_workers=num_workers)
  
    return data_loader

## Defining Autoencoder class

In [ ]:
class MTAutoEncoder(nn.Module):
    def __init__(self, num_inputs=990, 
                 num_latent=200, tied=True,
                 num_classes=2, use_dropout=False):
        super(MTAutoEncoder, self).__init__()
        
        self.num_latent = num_latent
        self.num_inputs = num_inputs
        
        self.fc_encoder = nn.Sequential (
                nn.Linear(self.num_inputs,4096),
                nn.Tanh(),
                nn.Linear(4096,1024),
                nn.Tanh())
        
        self.fc_decoder = nn.Sequential (
                nn.Linear(1024,4096),
                nn.Tanh(),
                nn.Linear(4096,self.num_inputs),
                nn.Tanh())
         
        
        if use_dropout:
            self.classifier = nn.Sequential (
                nn.Dropout(p=0.25),
                nn.Linear(1024, 1),
#                 nn.Sigmoid(),
#                 nn.Linear(128, 1),

            )
        else:
            self.classifier = nn.Sequential (
                nn.Linear(1024, 1),
#                 nn.Sigmoid(),
#                 nn.Linear(128, 1),
            )
            
         
    def forward(self, x, eval_classifier=False):

        if eval_classifier:
            x = self.fc_encoder(x)
            x_logit = self.classifier(x).squeeze()
            
        else:
            x = self.fc_encoder(x)
            x = self.fc_decoder(x)
            x_logit = None
            
        return x, x_logit

mtae = MTAutoEncoder()

mtae

## Defining training and testing functions

In [ ]:
def train(model, epoch, train_loader, p_bernoulli=None, mode='both', lam_factor=1.0):
    model.train()
    train_losses = []
    clf_train_loss = []
    ae_train_loss = []
    
    if mode == 'clf':
        final_targets = []
        final_predictions = []
    else:
        final_targets = None
        final_predictions = None    
    
    for i,(batch_x,batch_y) in enumerate(train_loader):
        if len(batch_x) != batch_size:
            continue
        if p_bernoulli is not None:
            if i == 0:
                p_tensor = torch.ones_like(batch_x).to(device)*p_bernoulli
            rand_bernoulli = torch.bernoulli(p_tensor).to(device)

        data, target = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()

        if mode == 'ae':
            if p_bernoulli is not None:
                rec_noisy, _ = model(data*rand_bernoulli, False)
                loss_ae = criterion_ae(rec_noisy, data) / len(batch_x)
            else:
                rec, _ = model(data, False)
                loss_ae = criterion_ae(rec, data) / len(batch_x)
                
            loss_total = loss_ae
            loss_ae_np = loss_ae.detach().cpu().numpy()
            
            clf_train_loss.append(0.0)
            ae_train_loss.append(loss_ae_np)
            train_losses.append([loss_ae_np, 0.0])
                
        if mode == 'clf':
            rec_clean, logits = model(data, True)
            loss_clf = criterion_clf(logits, target)
            
            proba = torch.sigmoid(logits).detach().cpu().numpy()
            predictions = np.ones_like(proba, dtype=np.int32)
            predictions[proba < 0.5] = 0
            
            final_targets.append(target.detach().cpu().numpy())
            final_predictions.append(predictions)
            
            loss_total = loss_clf
            loss_clf_np = loss_clf.detach().cpu().numpy()
            
            clf_train_loss.append(loss_clf_np)
            ae_train_loss.append(0.0)
            train_losses.append([0.0,loss_clf_np])
            

        loss_total.backward()
        optimizer.step()
    
    if (final_targets is not None) and (final_predictions is not None):
        final_targets = np.concatenate(final_targets)
        final_predictions = np.concatenate(final_predictions)
        train_accuracy = np.mean(final_targets == final_predictions)

        return np.mean(clf_train_loss), train_accuracy
    else:
        return np.mean(ae_train_loss), None

def test(model, criterion, test_loader, 
         eval_classifier=False, num_batch=None):
    test_loss, n_test, correct = 0.0, 0, 0
    eval_loss = []
    all_predss=[]
    if eval_classifier:
        y_true, y_pred = [], []
    with torch.no_grad():
        model.eval()
        for i,(batch_x,batch_y) in enumerate(test_loader):
            if num_batch is not None:
                if i >= num_batch:
                    continue
            data, target = batch_x.to(device), batch_y.to(device)
            rec, logits = model(data, eval_classifier)

#             test_loss += criterion(rec, data).detach().cpu().numpy() 
#             n_test += len(batch_x)
            test_loss = criterion(logits, target).detach().cpu().numpy() 
            eval_loss.append(test_loss)
            if eval_classifier:
                proba = torch.sigmoid(logits).detach().cpu().numpy()
                preds = np.ones_like(proba, dtype=np.int32)
                preds[proba < 0.5] = 0
                all_predss.extend(preds)###????
                y_arr = np.array(batch_y, dtype=np.int32)

                correct += np.sum(preds == y_arr)
                y_true.extend(y_arr.tolist())
                y_pred.extend(proba.tolist())
        mlp_acc,mlp_sens,mlp_spef = confusion(y_true,all_predss)
        metrics_dict = {'accuracy': np.round(mlp_acc, 4), 
                        'senstivity' : np.round(mlp_sens,4), 
                        'specificity' : np.round(mlp_spef,4), 
                        'loss' : np.round(np.mean(eval_loss),4)}
        
    return  metrics_dict

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
if p_Method == "ASD-DiagNet" and p_mode == "whole":
    
    start = time.time()
    batch_size = 16
    learning_rate_ae, learning_rate_clf = 0.0001, 0.0001
    num_epochs = 50

    p_bernoulli = None
    augmentation = p_augmentation
    use_dropout = True

    start= time.time()

#     print('p_bernoulli: ', p_bernoulli)
#     print('augmentaiton: ', augmentation, 'aug_factor: ', aug_factor, 
#           'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
#     print('use_dropout: ', use_dropout, '\n')
    
    # list to store metrics after each fold
    repeat_acc=[]
    repeat_sen=[]
    repeat_spec=[]
    repeat_loss=[]
    
    
    for rp in range(1):
        kf = StratifiedKFold(n_splits=p_fold, random_state=1, shuffle=True)
    # list to store metrics after each fold
        crossval_acc=[]
        crossval_sen=[]
        crossval_spec=[]
        crossval_loss=[]
        
        for kk,(train_index, test_index) in enumerate(kf.split(sfc, labels)):
            
            NAME = f'asd-diagnet-fold-{kk+1}-rp-2'
            ID = f'fold-{kk+1}-rp-2.1'
        
            x_train, y_train = sfc[train_index],labels[train_index]
            x_test, y_test = sfc[test_index],labels[test_index]


            verbose = (True if (kk == 0) else False)



            num_inpp = 19900
            n_lat = 512
            
            train_dataset = ASDDataset(x_train,y_train)
            test_dataset = ASDDataset(x_test,y_test)

            train_dataloader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
            test_dataloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)
               

            model = MTAutoEncoder(tied=False, num_inputs=num_inpp, num_latent=n_lat, use_dropout=use_dropout)
            model.to(device)

            criterion_ae = nn.MSELoss(reduction='sum')
            criterion_clf = nn.BCEWithLogitsLoss()
#             optimizer = optim.SGD([{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
#                                    {'params': model.fc_decoder.parameters(), 'lr': learning_rate_ae},
#                                    {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
#                                   momentum=0.9)
            
            optimizer = optim.Adam(model.parameters(),lr=0.0001,weight_decay=0.05)          



            for epoch in range(1, num_epochs+1):
                
                if epoch <= 30:
                    ae_train_loss,_ = train(model, epoch, train_dataloader, p_bernoulli, mode='ae')
                    content = f'AE Train loss: {(ae_train_loss):.4f}'

                else:
                    clf_train_loss, train_acc = train(model, epoch, train_dataloader, p_bernoulli, mode='clf')
                    content = f'CLF Train loss: {(clf_train_loss):.4f}, Train Accuracy: {(train_acc):.4f}'
                    
                print(f'Epoch {epoch}/{num_epochs+1}')
                print(content)

            metrics_dict = test(model, criterion_clf, test_dataloader, eval_classifier=True)
            print("-----------------------------")
            print(f'Fold {kk+1}/{p_fold}')
            content = f'{metrics_dict}'
            print(content)
            print("-----------------------------")
            
            crossval_acc.append(metrics_dict['accuracy'])
            crossval_sen.append(metrics_dict['senstivity'])
            crossval_spec.append(metrics_dict['specificity'])
            crossval_loss.append(metrics_dict['loss'])
            
            #save the model after each fold
            
            recorder = {'optimizer': optimizer.state_dict(),
            'model': model.state_dict(),
            'fold' : kk+1,
            'repitition' : rp+1}

            torch.save(recorder, f'{NAME}.pt')
            
        print("*********************************")    
        print(f'Average Value after 10 Folds and repeats one {rp+1}------->')
        content = f'Accuracy: {np.round(np.mean(crossval_acc),4)}, Senstivity: {np.round(np.mean(crossval_sen),4)}, Specificity: {np.round(np.mean(crossval_spec),4)}, Loss: {np.round(np.mean(crossval_loss),4)}'
        print(content)
        print("*********************************") 
        
        repeat_acc.append(np.mean(crossval_acc))
        repeat_sen.append(np.mean(crossval_sen))
        repeat_spec.append(np.mean(crossval_spec))
        repeat_loss.append(np.mean(crossval_loss))
    
    print(f"Average Value after 1 Repeat:")
    content = f'Accuracy: {np.round(np.mean(repeat_acc),4)}, Senstivity: {np.round(np.mean(repeat_sen),4)}, Specificity: {np.round(np.mean(repeat_spec),4)}, Loss: {np.round(np.mean(repeat_loss),4)}'
    print(content)
        
    finish= time.time()
    print(finish-start)




In [ ]:
Accuracy: 0.6724, Senstivity: 0.61, Specificity: 0.7321, Loss: 0.6234999895095825
589.3430550098419

In [ ]:
adam with weight decay 0.3 

Accuracy: 0.657, Senstivity: 0.5431, Specificity: 0.766, Loss: 0.6509000062942505
1160.326141834259


adam w/o weight decay 
Accuracy: 0.7043, Senstivity: 0.6636, Specificity: 0.7434, Loss: 0.5964999794960022
1056.0533220767975


adam with weight decay 0.1 
Average Value after 1 Repeat:
Accuracy: 0.7063, Senstivity: 0.6716, Specificity: 0.7396, Loss: 0.5821999907493591
1162.2546133995056

adam with weight decay 0.15 
Accuracy: 0.686, Senstivity: 0.6598, Specificity: 0.7113, Loss: 0.5911999940872192
1161.7236032485962

adam with weight decay 0.1  and dropout 0.25
Accuracy: 0.6985, Senstivity: 0.6793, Specificity: 0.717, Loss: 0.5842000246047974
1162.4763264656067


adam with weight decay 0.1  and dropout 0.25 n_lat 256
Average Value after 1 Repeat:
Accuracy: 0.6908, Senstivity: 0.6933, Specificity: 0.6887, Loss: 0.5952000021934509
1149.6250448226929

In [ ]:
AIMAFE Data


adam with weight decay 0.1  and dropout 0.25 n_lat 256
Average Value after 1 Repeat:
Accuracy: 0.7397, Senstivity: 0.6706, Specificity: 0.7944, Loss: 0.5264000296592712
1056.9974591732025


adam with weight decay 0.1  and dropout 0.25 n_lat 512
Average Value after 1 Repeat:
Accuracy: 0.7471, Senstivity: 0.6923, Specificity: 0.7906, Loss: 0.5105000138282776
1067.9337112903595

adam with weight decay 0.2  and dropout 0.25 n_lat 512
Average Value after 1 Repeat:
Accuracy: 0.7324, Senstivity: 0.6256, Specificity: 0.817, Loss: 0.5450999736785889
1068.7675378322601